### Assignment 3- Features Selection
Name: Vivian Lim Bi Fang 

ID: P101458

In [4]:
# import data 
import pandas as pd

# hotel_reviews_data is being preprocess by removing stopwords, punctuation and numbers in assignment 2.
hotel_reviews_data = pd.read_csv("Labelled_Hotel_Review_Data.csv")
hotel_reviews_data.head()

,Google_Translate,Clean_Text,Sentiment
0,Service is very friendly staff made ??us very ...,Service friendly staff made us ...,Positive
1,Our family stayed one night and found this hot...,family stayed one night found ...,Positive
2,Mid-sized room but clean. Good breakfast food ...,Midsized room clean Good breakfa...,Positive
3,"Hotel is on the top of a shopping Pavillion, L...",Hotel top shopping Pavillion Lot...,Positive
4,I am comfortable with accommodation in penang ...,comfortable accommodation penang ha...,Positive


In [5]:
# lemmatization 


from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize


def lemm(sentences):
    
    lemmatizer = WordNetLemmatizer()
    wordlist = word_tokenize(sentences)

    
    result = []
    for word in wordlist:
        output = lemmatizer.lemmatize(word, pos= "v")
        result.append(output)
       
    result =  " ".join(result)
    return result

hotel_reviews_data["Lemmatized"] = hotel_reviews_data.apply(lambda row : lemm(row["Clean_Text"]), axis=1)
hotel_reviews_data = hotel_reviews_data[['Lemmatized','Sentiment']]
hotel_reviews_data

,Lemmatized,Sentiment
0,Service friendly staff make us comfortable fam...,Positive
1,family stay one night find hotel much satisfac...,Positive
2,Midsized room clean Good breakfast food option...,Positive
3,Hotel top shop Pavillion Lot suitable couple w...,Positive
4,comfortable accommodation penang hard rock com...,Positive
...,...,...
93,park access card not use due fluctuations offi...,Negative
94,Cubicles kitchens excessive stink not directly...,Negative
95,Necessary provide sufficient area train layout...,Negative
96,everything ok except small rat chamber,Negative


In [6]:
# create CSV file for later use in assignment 4

hotel_reviews_data.to_csv(r'C:\Users\vivia\Documents\Data Science\Unstructure Data Analysis\Assignments & Project\assignment4.csv', header= True, index = False)


In [7]:
# check for data balanced to avoid bias 
# dataset is balanced. Not necessary to conduct bookstrapping

import matplotlib.pyplot as plt
import seaborn as sns

# sentiment_count = hotel_reviews_data["Sentiment"].value_counts()
# print (sentiment_count)
# plt.figure(figsize=(6,4))
# sns.barplot(sentiment_count.index, sentiment_count.values, alpha =0.5, )

sns.set_style('whitegrid')
sns.countplot(x='Sentiment',data=hotel_reviews_data,palette='rainbow')
plt.ylabel ("Frequency")
plt.xlabel("Sentiments")
plt.show()

<Figure size 640x480 with 1 Axes>

In [8]:
# split the data into training and test set 

from sklearn.model_selection import train_test_split

input_data = hotel_reviews_data["Lemmatized"].values
label_data = hotel_reviews_data["Sentiment"].values

x_train, x_test, y_train, y_test = train_test_split(input_data, label_data, test_size = 0.2, random_state=42)
print (len(x_train))
print (len(x_test))
print (len(y_train))
print (len(y_test))

# x_train

78
20
78
20


# BAG OF WORDS

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

BOW = CountVectorizer()
BOW.fit(x_train)
BOW_train = BOW.transform(x_train)
# print (BOW_train)
BOW_test = BOW.transform(x_test)

print ((BOW_train.shape[1]))
print ((BOW_test.shape[1]))

# BOW_train
# print(BOW_train[0:2])

735
735


In [10]:
word_BOW = pd.DataFrame(BOW_train.toarray(), columns=BOW.get_feature_names())
word_BOW.head()


# print(word_BOW[0:10])
# print(BOW_train[0:2].toarray()

,access,accessible,accommodation,accommodations,accordance,according,active,activities,add,addition,...,work,workers,world,worth,worthwhile,would,year,yet,yg,yummy
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
# test on NB 

from sklearn.naive_bayes import MultinomialNB

NB_BOW = MultinomialNB()
NB_BOW.fit(BOW_train, y_train)
predict_NB_BOW = NB_BOW.predict(BOW_test)


In [12]:
# test for F1 score, precision and recall 
from sklearn.metrics import classification_report

result_BOW = classification_report(predict_NB_BOW, y_test)
print (result_BOW)

              precision    recall  f1-score   support

    Negative       0.88      1.00      0.93         7
    Positive       1.00      0.92      0.96        13

    accuracy                           0.95        20
   macro avg       0.94      0.96      0.95        20
weighted avg       0.96      0.95      0.95        20



# TF-IDF

Study the accuracy of TF-IDF using unigram and compare with ngram range from 1 to 3.


In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

def TFIDF_ngram (n_gram, x_train, y_train, x_test, y_test):
    
    TFIDF = TfidfVectorizer(analyzer = "word", ngram_range = n_gram)
    TFIDF.fit (x_train)
    TFIDF_train = TFIDF.transform (x_train)
    TFIDF_test = TFIDF.transform (x_test)
#     word_TFIDF = pd.DataFrame(TFIDF_train.toarray(), columns = TFIDF.get_feature_names())
#     word_TFIDF
    
#     print (TFIDF_train)
    NB_TFIDF = MultinomialNB()
    NB_TFIDF.fit(TFIDF_train, y_train)
    predict_NB_TFIDF = NB_TFIDF.predict(TFIDF_test)
#     print (predict_NB_TFIDF)
    
    print ("")
    print ("ngram =" +str(n))

    result_TFIDF = classification_report(predict_NB_TFIDF, y_test)
    print (result_TFIDF)

    return (result_TFIDF)

In [14]:
ngram = [(1,1), (2,2), (3,3), (4,4)]
# ngram = [(1,1), (1,2), (1,3), (1,4)]
for n in ngram:
    TFIDF_ngram (n, x_train, y_train, x_test, y_test)
#     print (n)


ngram =(1, 1)
              precision    recall  f1-score   support

    Negative       0.88      1.00      0.93         7
    Positive       1.00      0.92      0.96        13

    accuracy                           0.95        20
   macro avg       0.94      0.96      0.95        20
weighted avg       0.96      0.95      0.95        20


ngram =(2, 2)
              precision    recall  f1-score   support

    Negative       0.62      0.56      0.59         9
    Positive       0.67      0.73      0.70        11

    accuracy                           0.65        20
   macro avg       0.65      0.64      0.64        20
weighted avg       0.65      0.65      0.65        20


ngram =(3, 3)
              precision    recall  f1-score   support

    Negative       1.00      0.44      0.62        18
    Positive       0.17      1.00      0.29         2

    accuracy                           0.50        20
   macro avg       0.58      0.72      0.45        20
weighted avg       0.92      

C:\Users\vivia\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


Conclude that, direct comparison of unigram to bigram, trigram is not encourage. This is because forming exactly bigram / trigram sequence increase sparsing thus reduce efficiecy. Besides, finding exactly the word sequece (x,x1) / (x,x2,x3) would be very hard which result the worst accuracy. 

Further approach can be tried using different ngram_range such as ngram = [(1,1), (1,2), (1,3), (1,4)]. Surprising, the result are the same thus, unigram is sufficient for this assignment. 

# Word Embedding (Doc2Vec)

In [15]:
# create new training and test set for tagged documents purpose

Doc2vec_data = hotel_reviews_data[["Lemmatized", "Sentiment"]]
# print (Doc2vec_data.head())


train, test = train_test_split(Doc2vec_data, test_size = 0.2, random_state=42)


In [18]:
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
import nltk
nltk.download('punkt')
from sklearn import utils

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vivia\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [21]:
def tokenize_text(sentence):
    tokens = []
#     print (sentence)
    for sent in nltk.sent_tokenize(sentence):
        for word in nltk.word_tokenize(sent):
#             print (word)
#             print (len(word))
            if len(word) < 2:
                continue
            tokens.append(word.lower())
    return tokens

# tagged the documents with unique ID
train_tagged = train.apply(lambda x : TaggedDocument(words = tokenize_text(x["Lemmatized"]),tags=[x.Sentiment]), axis=1)
# print (train_tagged.values)
test_tagged = test.apply(lambda x : TaggedDocument(words = tokenize_text(x["Lemmatized"]),tags=[x.Sentiment]), axis=1)

In [22]:
# build vocab

model_dbow = Doc2Vec(dm=0, vector_size=300, negative =5, hs=0, min_count =1, sample=0, workers = 8, alpha=0.025, min_alpha=0.001)
model_dbow.build_vocab([x for x in train_tagged.values])


In [23]:
# train model

for epoch in range (50):
    model_dbow.train(utils.shuffle([x for x in train_tagged.values]), total_examples = len(train_tagged.values), epochs =1)
    model_dbow.alpha -= 0.002
    model_dbow.min_alpha = model_dbow.alpha

In [24]:
# create final vector feature
# infer_vector to create document vectors for another sample corpus

def vec_for_learning(model, tagged_docs):
    sents = tagged_docs.values
    targets, regressors = zip(*[(doc.tags[0], model.infer_vector(doc.words, steps=20)) for doc in sents])
#     print (targets, regressors)
    return targets, regressors

Doc2Vec_y_train, Doc2Vec_x_train = vec_for_learning(model_dbow, train_tagged)
# print (Doc2Vec_x_train)
Doc2Vec_y_test, Doc2Vec_x_test = vec_for_learning(model_dbow, test_tagged)

In [25]:
#  run in classifier

from sklearn.naive_bayes import GaussianNB #dealing with real-values features so use Gaussian NB
from sklearn.metrics import classification_report

NB_Doc2Vec = GaussianNB()
NB_Doc2Vec.fit(Doc2Vec_x_train, Doc2Vec_y_train)
predict_NB_Doc2Vec_train = NB_Doc2Vec.predict(Doc2Vec_x_train)
predict_NB_Doc2Vec_test = NB_Doc2Vec.predict(Doc2Vec_x_test)


# do this step for model parameter tuning
result_Doc2Vec_train = classification_report(predict_NB_Doc2Vec_train, Doc2Vec_y_train)
print("Training set prediction:")
print (result_Doc2Vec_train)

result_Doc2Vec_test = classification_report(predict_NB_Doc2Vec_test, Doc2Vec_y_test)
print("Testing set prediction:")
print (result_Doc2Vec_test)

Training set prediction:
              precision    recall  f1-score   support

    Negative       0.79      0.78      0.78        40
    Positive       0.77      0.79      0.78        38

    accuracy                           0.78        78
   macro avg       0.78      0.78      0.78        78
weighted avg       0.78      0.78      0.78        78

Testing set prediction:
              precision    recall  f1-score   support

    Negative       0.75      0.60      0.67        10
    Positive       0.67      0.80      0.73        10

    accuracy                           0.70        20
   macro avg       0.71      0.70      0.70        20
weighted avg       0.71      0.70      0.70        20



# Conclusion

From the classification reports (precision, recall, f1-score) shown above, 

1) TF-IDF (unigram) & BOW has the same accuracy which is 0.96, 0.95, 0.95 respectively.

2) Doc2Vec has lower accuracy (0.74, 0.65, 0.65) which might due to presence of noises or parameter tuning issues.